In [ ]:
import os
from ase.io import read, write

from ase.neb import NEB
from ase.optimize import BFGS, FIRE

In [ ]:
initial_image_path = 'struct/'
final_image_path = 'struct/'

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 10 /opt/chemistry/conquest/dev/bin/Conquest'
os.environ['CQ_PP_PATH'] = '/opt/chemistry/conquest/dev/pseudo-and-pao/'
os.environ['CQ_GEN_BASIS_CMD'] = 'mpirun -np 1 /opt/chemistry/conquest/dev/bin/MakeIonFiles'

In [ ]:
xc         = 'PBE'
basis_size = 'medium'
cutoff = 100.0
kpoints = [4,4,1]

In [ ]:
# basis = {
#          'Cu': {'file'            : 'Cu_PBE.ion'},
#          'O' : { 'file'            : 'O_PBE.ion' },             
#          'H' : { 'file'            : 'H_PBE.ion'},
#          'C' : { 'file'            : 'C_PBE.ion' },         
#           }


basis = {
         'Cu': {'gen_basis'            : True,
                'basis_size'           : basis_size,
                "pseudopotential_type": 'hamann',
                'xc'                   : xc,
                },
         'O' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },             
         'H' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },
         'C' : { 'gen_basis'            : True,
                 'basis_size'           : basis_size,
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : xc },         
          }

In [ ]:
from ase.calculators.conquest import Conquest

conquest_flags = {'IO.WriteOutToASEFile': True}

# Flags for atomic positions optimisation
conquest_flags.update({'AtomMove.TypeOfRun'   : 'static',  # optimization algorithm
                       'AtomMove.MaxForceTol' :  5e-3,   # max Force component in Ha/bohr                       
                       'AtomMove.ReuseDM'     :  True,    
                       'AtomMove.AppendCoords':  True,
                       'AtomMove.WriteXSF'  :  True,
                       })   

In [ ]:
# Set initial/final structures
initial = read(initial_image_path + 'ini.traj') # Use .traj file (not .xsf file) because it contains energy information of the structure
final = read(final_image_path + 'fin.traj')

In [ ]:
# Adjust the position of the structures
thr = 0.01



atom_exceed_x = [i for i in range(len(initial.positions)) if float(initial.positions[i,0]) > initial.cell[0,0]-thr]
for i in atom_exceed_x:
    initial.positions[i,0] = initial.positions[i,0] -initial.cell[0,0]
    
atom_exceed_y = [i for i in range(len(initial.positions)) if float(initial.positions[i,1]) > initial.cell[1,1]-thr]
for i in atom_exceed_y:
    initial.positions[i,1] = initial.positions[i,1] -initial.cell[1,1]

atom_exceed_x = [i for i in range(len(final.positions)) if float(final.positions[i,0]) > final.cell[0,0]-thr]
for i in atom_exceed_x:
    final.positions[i,0] = final.positions[i,0] -final.cell[0,0]
    
atom_exceed_y = [i for i in range(len(final.positions)) if float(final.positions[i,1]) > final.cell[1,1]-thr]
for i in atom_exceed_y:
    final.positions[i,1] = final.positions[i,1] -final.cell[1,1]

In [ ]:
# Set number of image between initial and final
n_img = 3

In [ ]:
# Make a band consisting of 5 images:
images = [initial]
images += [initial.copy() for i in range(n_img)]
images += [final]

In [ ]:
for image in images:
    image.calc = Conquest(directory      = f'neb_img',
                          grid_cutoff    = cutoff,
                          scf_tolerance=5.0e-8,  # Ha
                          self_consistent= True,
                          xc    = xc,
                          basis = basis,
                          kpts  = kpoints,
                          label = f'NEB_calc',
                          nspin = 1,
                          **conquest_flags)

In [ ]:
# Set neb
neb = NEB(images, climb=True)

# Interpolate linearly the potisions of the three middle images:
neb.interpolate()

In [ ]:
# Start calculation
optimizer = BFGS(neb, trajectory='neb_BFGS.traj')
optimizer.run(fmax=5e-3) # fmax should be same value for optimization of initial/final structure.